In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model, Sequential


2025-11-15 14:58:27.504238: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-15 14:58:27.520666: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 14:58:28.123666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-15 14:58:30.348561: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [3]:
# Build VGG16 model
inputs = Input(shape=(32, 32, 3))
conv1  = Conv2D(filters=64, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(inputs)
conv2  = Conv2D(filters=64, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, use_bias=False, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(512, use_bias=False, activation="relu")(flat) #Changeed from 4096 for true VGG16 to better fit CIFAR10
dense2 = Dense(512, use_bias=False, activation="relu")(dense1) #Changeed from 4096 for true VGG16 to better fit CIFAR10
output = Dense(10, use_bias=False, activation="softmax")(dense2) #10 classes in CIFAR10, changed from 1000 for true VGG16

model  = Model(inputs=inputs, outputs=output)
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2025-11-15 14:58:38.836574: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 128)    │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 256)      │       589,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 256)      │       589,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 512)      │     1,179,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 4, 4, 512)      │     2,359,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 4, 4, 512)      │     2,359,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 2, 2, 512)      │     2,359,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         5,120 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,239,872 (58.14 MB)

 Trainable params: 15,239,872 (58.14 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=100,                            # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v15_vgg16.keras")

Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 297s 839ms/step - accuracy: 0.0980 - loss: 2.3026 - val_accuracy: 0.1058 - val_loss: 2.3026
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 295s 839ms/step - accuracy: 0.1110 - loss: 2.3026 - val_accuracy: 0.1160 - val_loss: 2.3026
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 343s 974ms/step - accuracy: 0.1187 - loss: 2.3026 - val_accuracy: 0.1192 - val_loss: 2.3026
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 355s 1s/step - accuracy: 0.1223 - loss: 2.3026 - val_accuracy: 0.1210 - val_loss: 2.3026
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 353s 1s/step - accuracy: 0.1240 - loss: 2.3026 - val_accuracy: 0.1224 - val_loss: 2.3026
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 343s 975ms/step - accuracy: 0.1281 - loss: 2.3026 - val_accuracy: 0.1262 - val_loss: 2.3025
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 353s 1s/step - accuracy: 0.1308 - loss: 2.3025 - val_accuracy: 0.1302 - val_loss: 2.3025
Epoch 8/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 361s 1s/step - accuracy: 0.1333 - los

In [5]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [6]:
net = snn_converter.load_keras_model("CIFAR_v15_vgg16.keras", show_info=True)

* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 64) 
* Conv2D layer: conv2d_1, (32, 32, 64) 
* MaxPooling2D layer: max_pooling2d, (16, 16, 64) 
* Conv2D layer: conv2d_2, (16, 16, 128) 
* Conv2D layer: conv2d_3, (16, 16, 128) 
* MaxPooling2D layer: max_pooling2d_1, (8, 8, 128) 
* Conv2D layer: conv2d_4, (8, 8, 256) 
* Conv2D layer: conv2d_5, (8, 8, 256) 
* Conv2D layer: conv2d_6, (8, 8, 256) 
* MaxPooling2D layer: max_pooling2d_2, (4, 4, 256) 
* Conv2D layer: conv2d_7, (4, 4, 512) 
* Conv2D layer: conv2d_8, (4, 4, 512) 
* Conv2D layer: conv2d_9, (4, 4, 512) 
* MaxPooling2D layer: max_pooling2d_3, (2, 2, 512) 
* Conv2D layer: conv2d_10, (2, 2, 512) 
* Conv2D layer: conv2d_11, (2, 2, 512) 
* Conv2D layer: conv2d_12, (2, 2, 512) 
* MaxPooling2D layer: max_pooling2d_4, (1, 1, 512) 
* Flatten skipped.
* Dense layer: dense, 512 
    weights: (512, 512)
    mean 0.0007144721457734704, std 0.04469093680381775
    min -0.09196483343839645, max 0.095

In [7]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [4:14:11<00:00, 50.84s/it]  


In [13]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00        24
           2       0.08      0.30      0.12        27
           3       0.00      0.00      0.00        29
           4       0.00      0.00      0.00        23
           5       0.06      0.25      0.10        28
           6       0.11      0.26      0.16        34
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        37
           9       0.00      0.00      0.00        35

    accuracy                           0.08       300
   macro avg       0.02      0.08      0.04       300
weighted avg       0.03      0.08      0.04       300

Test accuracy of the SNN: 0.08


/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

In [9]:
model = tf.keras.models.load_model('CIFAR_v15_vgg16.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 80ms/step - accuracy: 0.6588 - loss: 3.2132


In [10]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.6588
Test accuracy of the SNN: 0.08
